In [92]:
import pandas as pd
import os 

In [ ]:
rutaJson = "D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Florida/"     # Ruta de los json de Sitios
archivos_json = [os.path.join(rutaJson, archivo) for archivo in os.listdir(rutaJson) if archivo.endswith('.json')]
archivos_json


In [3]:
df_estados_USA= pd.read_csv("D:/PROYECTO FINAL/Google Maps/estados_usa.csv",delimiter = ';', encoding = "utf-8")

In [107]:
df_reviews= pd.read_json('D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Florida/1.json', lines=True)

In [109]:
df_reviews.head(3)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.014719e+20,Julie A. Gerber,1628003250740,1,Update: Their “reply” to my review amounted to...,None,"{'time': 1627042799532, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.154772e+20,Martin Sheffield,1595031217005,5,He's a knowledgeable doctor but the way he run...,None,"{'time': 1582464056733, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
2,1.018050e+20,Brian Truett,1522924253567,5,"Best doctor I've ever had, I never wait to be ...",None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb


In [110]:
df_reviews.isna().sum()

user_id         0
name            0
time            0
rating          0
text        49977
pics       145423
resp       118033
gmap_id         0
dtype: int64

In [111]:
df_reviews['time'] = pd.to_datetime(df_reviews['time'],unit='ms')     #Transforma la columna de tipo timestamp a formato datetime

In [112]:
print(df_reviews.shape)
df_reviews.head()

(150000, 8)


,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.014719e+20,Julie A. Gerber,2021-08-03 15:07:30.740,1,Update: Their “reply” to my review amounted to...,None,"{'time': 1627042799532, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.154772e+20,Martin Sheffield,2020-07-18 00:13:37.005,5,He's a knowledgeable doctor but the way he run...,None,"{'time': 1582464056733, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
2,1.018050e+20,Brian Truett,2018-04-05 10:30:53.567,5,"Best doctor I've ever had, I never wait to be ...",None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
3,1.063444e+20,Tina Sun,2016-07-07 16:10:19.586,1,I was told he is a good doctor. I was trying t...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,1.008751e+20,James Haynes,2016-12-02 12:56:55.081,5,Takes the time to actually get to know his pat...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb


In [113]:
df_reviews = df_reviews[(df_reviews['time'].dt.year >= 2010) & (df_reviews['time'].dt.year <= 2021)]

In [114]:
# Se guardan las columnas resp y el gmap_id en un df
df_respuestas= df_reviews[['gmap_id','resp']]
df_respuestas= df_respuestas.dropna(subset=['resp']) # Elimina filas con resp nulo
df_respuestas= df_respuestas.rename(columns={'gmap_id': 'business_id'})


In [115]:
df_respuestas.head()

,gmap_id,resp
0,0x8893863ea87bd5dd:0x9383ebf973e74abb,"{'time': 1627042799532, 'text': 'Thank you for..."
1,0x8893863ea87bd5dd:0x9383ebf973e74abb,"{'time': 1582464056733, 'text': 'Thank you for..."
15,0x88c33c2935058dd3:0xb888c702ecf5aef6,"{'time': 1533077814863, 'text': 'Thank you for..."
17,0x88c33c2935058dd3:0xb888c702ecf5aef6,"{'time': 1542812762776, 'text': 'Thank you so ..."
18,0x88c33c2935058dd3:0xb888c702ecf5aef6,"{'time': 1533077488025, 'text': 'Thank you Jua..."


In [116]:
# Funcion para cargar los json a una serie de pandas
def expandir_diccionario(diccionario):
    return pd.Series(diccionario)

# Se concatenas las series al df original
df_expandido = pd.concat([df_respuestas, df_respuestas['resp'].apply(expandir_diccionario)], axis=1)
df_expandido.drop('resp', axis=1, inplace=True)     # Se elimina la columna de MISC
df_expandido['time'] = pd.to_datetime(df_expandido['time'],unit='ms')     #Transforma la columna de tipo timestamp a formato datetime

In [117]:
df_respuestas= df_expandido
df_respuestas.head(3)

,gmap_id,time,text
0,0x8893863ea87bd5dd:0x9383ebf973e74abb,2021-07-23 12:19:59.532,Thank you for the feedback. We would like to w...
1,0x8893863ea87bd5dd:0x9383ebf973e74abb,2020-02-23 13:20:56.733,Thank you for reaching out. I’m forwarding you...
15,0x88c33c2935058dd3:0xb888c702ecf5aef6,2018-07-31 22:56:54.863,Thank you for the kind review Cynthia!


In [118]:
df_reviews= df_reviews.drop(['pics','resp','name'], axis=1)       #Elimina la columna pics
df_reviews= df_reviews.rename(columns={'time': 'date', 'gmap_id':'business_id', 'rating':'stars'})

In [119]:
df_reviews[['useful','funny','cool']] = 0
df_reviews['platform']= 1

In [120]:
df_reviews= df_reviews[['business_id','user_id', 'date', 'stars','useful','funny','cool', 'text', 'platform']]

In [126]:
df_reviews['user_id']= df_reviews['user_id'].astype(str)

In [127]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149993 entries, 0 to 149999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   business_id  149993 non-null  object        
 1   user_id      149993 non-null  object        
 2   date         149993 non-null  datetime64[ns]
 3   stars        149993 non-null  int64         
 4   useful       149993 non-null  int64         
 5   funny        149993 non-null  int64         
 6   cool         149993 non-null  int64         
 7   text         100016 non-null  object        
 8   platform     149993 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 11.4+ MB


In [128]:
df_reviews

,business_id,user_id,date,stars,useful,funny,cool,text,platform
0,0x8893863ea87bd5dd:0x9383ebf973e74abb,1.0147185615514873e+20,2021-08-03 15:07:30.740,1,0,0,0,Update: Their “reply” to my review amounted to...,1
1,0x8893863ea87bd5dd:0x9383ebf973e74abb,1.1547723478903833e+20,2020-07-18 00:13:37.005,5,0,0,0,He's a knowledgeable doctor but the way he run...,1
2,0x8893863ea87bd5dd:0x9383ebf973e74abb,1.0180501024489284e+20,2018-04-05 10:30:53.567,5,0,0,0,"Best doctor I've ever had, I never wait to be ...",1
3,0x8893863ea87bd5dd:0x9383ebf973e74abb,1.0634442288149375e+20,2016-07-07 16:10:19.586,1,0,0,0,I was told he is a good doctor. I was trying t...,1
4,0x8893863ea87bd5dd:0x9383ebf973e74abb,1.0087511306956177e+20,2016-12-02 12:56:55.081,5,0,0,0,Takes the time to actually get to know his pat...,1
...,...,...,...,...,...,...,...,...,...
149995,0x88ecf453fa2e0e71:0x3e7cc80808a41906,1.0713490855400841e+20,2018-11-03 20:35:58.844,4,0,0,0,Nice place to live. Mostly quite. Not great fo...,1
149996,0x88ecf453fa2e0e71:0x3e7cc80808a41906,1.003228202831872e+20,2018-08-21 19:11:38.516,1,0,0,0,Go leave reviews on their Facebook too. They h...,1
149997,0x88ecf453fa2e0e71:0x3e7cc80808a41906,1.1550430881276879e+20,2014-09-10 04:14:14.247,1,0,0,0,DO NOT MOVE HERE! I've been to the office coun...,1
149998,0x88ecf453fa2e0e71:0x3e7cc80808a41906,1.0929876385816103e+20,2015-06-28 13:56:06.210,2,0,0,0,"Alright first things first, if you're looking ...",1
